In [32]:
#뭔가 복잡쓰... 헤헤 
import tensorflow as tf
import numpy as np
from glob import glob
from PIL import Image

def get_label_from_path(path):
    return path.split('/')[-2]

def read_image(path):
    image = np.array(Image.open(path))
    return image.reshape(image.shape[0], image.shape[1], 1)

data_list = glob('/Users/jeong-kyujin/desktop/data_dir/*/*.jpg')

label_name_list = []
for path in data_list:
    label_name_list.append(get_label_from_path(path))

unique_label_names = np.unique(label_name_list)

def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path((path))
    onehot_label = onehot_label.astype(np.unit8)
    return onehot_label
    
def _read_py_function(path, label):
    image = read_image(path)
    label = np.array(label, dtype=np.unit8)
    return image.astype(np.int32), label

def _resize_function(image_decoded, label):
    image_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize_images(image_decoded, [28, 28])
    return image_resized, label

label_list = [onehot_encode_label(path).tolist()
              for path in data_list]

dataset = tf.contrib.data.Dataset.from_tensor_slices((data_list, label_list))
dataset = dataset.map(lambda data_list, label_list: tuple(tf.py_func(_read_py_function, 
                      [data_list, label_list], [tf.int32, tf.unit8])))
dataset = dataset.repeat(10)
dataset = dataset.shuffle(buffer_size=(int(len(data_list)*0.4)+3 * batch_size))
dataset = dataset.batch(batch_size)

iterator = dataset.make_initializable_iterator()
image_stacked, label_stacked = iterator.get_next()

next_element = iterator.get_next()

with tf.Session() as sess:
    sess.run(iterator.initializer)
    image, label = sess.run([image_stacked, label_stacked])

AttributeError: module 'numpy' has no attribute 'unit8'

In [17]:
#mnist 코드 응용해서 내 이미지를 이용한 학습
#코드 참고 : https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-11-4-mnist_cnn_layers.py
#과제 : 데이터 학습가능한 포맷으로 만들어서 넘겨줘야 함

import os
from glob import glob
from PIL import Image
import numpy as np
import tensorflow as tf

tf.set_random_seed(777)

# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

learning_rate = 0.001
training_epochs = 5
batch_size = 64

class Model:
    
    def __init__(self, sess, name):
        self.sess = sess
        self.name = name
        self._build_net()
        
    def _build_net(self):
        with tf.variable_scope(self.name):
            
            self.training = tf.placeholder(tf.bool)
            #input place holders
            self.X = tf.placeholder(tf.float32, [None, 784])
            
            #img 28x28x1 (black&white), Input layer
            X_img = tf.reshape(self.X, [-1, 28, 28, 1])
            self.Y = tf.placeholder(tf.float32, [None, 10])
            
            conv1 = tf.layers.conv2d(inputs=X_img, filters=32,kernel_size=[3,3],
                                    padding="SAME", activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[4,4],
                                          padding="SAME", strides=4)
            dropout1 = tf.layers.dropout(inputs=pool1, rate=0.3, training=self.training)

            conv2 = tf.layers.conv2d(inputs=dropout1, filters=64, kernel_size=[3,3],
                                    padding="SAME", activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[4,4],
                                           padding="SAME", strides=4)
            dropout2 = tf.layers.dropout(inputs=pool2, rate=0.3, training=self.training)
            
            conv3 = tf.layers.conv2d(inputs=dropout2, filters=128, kernel_size=[3,3],
                                    padding="SAME", activation=tf.nn.relu)
            pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=[2,2],
                                           padding="SAME", strides=2)
            dropout3 = tf.layers.dropout(inputs=pool3, rate=0.3, training=self.training)
            
            flat = tf.reshape(dropout3, [-1, 128 * 4 * 4])
            dense4 = tf.layers.dense(inputs=flat, uints=625, activation=tf.nn.relu)
            dropout4 = tf.layers.dropout(inputs=dense4, rate=0.5, training=self.training)
            
            self.logits = tf.layers.dense(inputs=dropout4, uints=10)
            
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
        logits=self.logits, labels=self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(self.cost)
        
        correct_prediction = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, training=False):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.training=training})
    
    def get_accuracy(self, x_test, y_test, training=False):
        return self.sess.run(self.accuracy,
                            feed_dict={self.X: x_test, self.Y: y_test,
                                      self.training: training})
    def train(self, x_data, y_data, training=True):
        return self.sess.run([self.cost, self.optimizer], feed_dict={
            self.X: x_data, self.Y: y_data, self.training: training})
    
#initialize
sess = tf.Session()
m1 = Model(sess, "m1")

sess.run(tf.global_variables_initializer())

print('Learning Started!')

#train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(mnist.train.num_examples / batch_size)
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        c, _ = m1.train(batch_xs, batch_ys)
        avg_cost += c / total_batch
        
    print('Epoch:', '%04d' % (epoch + 1), 'cost=', '{:.9f}'.format(avg_cost))
    
print('Learning Finished!')

print('Accuracy:', m1.get_accuracy(mnist.test.images, mnist.test.lables))



TypeError: 'Tensor' object is not iterable.

In [ ]:
import tensorflow as tf


data_list = tf.random_shuffle.glob('/Users/jeong-kyujin/desktop/data_dir/*/*.jpg')

for path in data_list:
    image = np.array(Image.open(path))
    X_img = image.reshape(image.shape[0], image.shape[1], 1)
    Y = int(path.split('/')[-2])

In [5]:

#이미지 주소로부터 데이터 추출 (이미지 & 라벨)
data_list = glob('/Users/jeong-kyujin/desktop/data_dir/*/*.jpg')

def get_label_from_path(path):
    return int(path.split('/')[-2])

image = np.array(Image.open(path))

def read_image(path):
    image = np.array(Image.open(path))
    return image.reshape(image.shape[0], image.shape[1], 1)
    #이건 흑백으로 만들어주는 거 같음! 원래 상태가 채널 3이니까, 다음에는 리셰잎없이 3으로!

class_name = get_label_from_path(path)

label_name_list = []
for path in data_list:
    label_name_list.append(get_label_from_path(path))
    
unique_label_names = np.unique(label_name_list)

def onehot_encode_label(path):
    onehot_label = unique_label_names == get_label_from_path(path)
    onehot_label = onehot_label.astype(np.uint8)
    return onehot_label

#hyper parameter

data_height = 128
data_width = 128
channel_n = 3

num_classes = 10

batch_image = np.zeros((batch_size, data_height, data_width, channel_n))
batch_label = np.zeros((batch_size, num_classes))

for n, path in enumerate(data_list[:batch_size]):
    image = read_image(path)
    onehot_label = onehot_encode_label(path)
    batch_image[n, :, :, :] = image
    batch_label[n, :] = onehot_label
    
#print(batch_image.shape, batch_label.shape)
    
batch_per_epoch = batch_size // len(data_list)

for batch_n in range(batch_per_epoch):
    batch_data = data_list[batch_n * batch_size : (batch_n + 1) * batch_size]

<TFRecordDataset shapes: (), types: tf.string>

<TensorSliceDataset shapes: (2,), types: tf.float64>
